# Data Wrangling Template

## Gather

In [2]:
import pandas as pd
import requests
import tweepy
import json
import time
import shutil

consumer_key = %env TWITTER_CON_KEY
consumer_secret = %env TWITTER_CON_SECRET
access_token = %env TWITTER_ACC_TOKEN
access_secret = %env TWITTER_ACC_SECRET

In [3]:
tw_arch = pd.read_csv('data/twitter-archive-enhanced.csv')

In [4]:
tsv_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

with requests.get(tsv_url, stream=True) as r:
    with open('data/image_predictions.tsv', 'wb') as f:
        shutil.copyfileobj(r.raw, f)

In [5]:
tw_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [6]:
tweet_ids = tw_arch.tweet_id.tolist()
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
with open('data/tweet-json.txt', mode='w') as f:
    for tid in tweet_ids:
        try:
            data = api.get_status(tid, tweet_mode='extended')
        except:
            print("Failure to retrieve tweet with tid: {0}".format(tid))
            continue
        f.write(json.dumps(data._json))
        time.sleep(0.3)

Failure to retrieve tweet with tid: 888202515573088257
Failure to retrieve tweet with tid: 873697596434513921
Failure to retrieve tweet with tid: 872668790621863937
Failure to retrieve tweet with tid: 872261713294495745
Failure to retrieve tweet with tid: 869988702071779329
Failure to retrieve tweet with tid: 866816280283807744
Failure to retrieve tweet with tid: 861769973181624320
Failure to retrieve tweet with tid: 856602993587888130
Failure to retrieve tweet with tid: 851953902622658560
Failure to retrieve tweet with tid: 845459076796616705
Failure to retrieve tweet with tid: 844704788403113984
Failure to retrieve tweet with tid: 842892208864923648
Failure to retrieve tweet with tid: 837366284874571778
Failure to retrieve tweet with tid: 837012587749474308
Failure to retrieve tweet with tid: 829374341691346946
Failure to retrieve tweet with tid: 827228250799742977
Failure to retrieve tweet with tid: 812747805718642688
Failure to retrieve tweet with tid: 802247111496568832
Failure to

Rate limit reached. Sleeping for: 346


Failure to retrieve tweet with tid: 754011816964026368
Failure to retrieve tweet with tid: 680055455951884288


Rate limit reached. Sleeping for: 447



   ## Assess

In [4]:
tw_arch.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


## Clean

#### Define

#### Code

#### Test